# Data Science: Bridging Principles and Practice
## Part 9: Machine Learning with Scikit-Learn

<img src="images/office-and-workers-in-barcelona-spain.jpg" />

<br>

*In this notebook, we will walk through solving a classification problem using machine learning. To do so, we will introduce the Scikit-Learn machine learning library for Python.*

### Table of Contents

<a href="#sectioncase">Case Study: Employee Attrition at IBM</a>

<ol start="9">
    <li><a href="#section9">Machine Learning</a>
        <ol type=a>
            <br>
            <li><a href="#section9a">The K-Nearest Neighbors Algorithm</a></li>
            <br>
            <li><a href="#section9b">Using Scikit-Learn: An Example</a></li>
            <br>
            <li><a href="#section9c">Using Scikit-Learn: KNN</a></li>            
        </ol>
    </li>
    </ol>

In [ ]:
# run this cell to import some necessary software
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import sklearn
import seaborn as sns
from gofer.ok import check
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D 

# set the random seed for reproducibility
np.random.seed(28)

## Case Study: Employee Attrition at IBM <a id= "sectioncase"></a>

*Employee attrition* refers to a decrease in employees at a company where the company does not intend to fill the vacancy afterward. Attrition, caused by resignations, retirements, or the elimination of job positions, is important for businesses to anticipate so they can keep costs low and properly distribute workloads.

Today, we'll try to predict whether or not an employee will leave their job using [a data set created by IBM](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/home). Because employee data is confidential, IBM's data set is fictional- it does not represent real employees. However, the data and the problem are structured very similarly to how actual companies such as IBM are approaching this very real problem.

<div class="alert alert-info">
    <b>NOTE</b>: we've made some changes to the data set to make this assignment simpler- things like renaming values, dropping confusing features, and adding definitions for columns. Please refer to the data dictionary below and <b>not</b> the one at the link when completing this assignment.
    </div>

Run the following cell to load the data.

In [ ]:
# load the data
attrition = pd.read_csv("data/attrition-renamed.csv", index_col="EmployeeNumber")
# show the first 5 rows
attrition.head()


This data set contains a wealth of information that could be used to predict attrition. Here's the **data dictionary**: the information about each feature (column) contained in the data set.


* `Age`: employee's age
* `Attrition`: the **response variable**. 1 if employee left IBM due to normal life circumstances, 0 if the employee stayed with IBM
* `BusinessTravel`: the amount of travel the employee does for business. 0 (no travel), 1 (rare travel), or 2 (frequent travel)
* `Department`: employee's home department. 'Sales', 'Research & Development', or 'Human Resources'
* `DistanceFromHome`: number of miles the employee has to travel to work
* `Education`: employee's highest level of education. 1 (below college), 2 (some college), 3 (bachelor's degree), 4 (master's degree), 5 (doctorate)
* `Gender`: 1 (female) or 0 (male)
* `JobRole`: 'Sales Executive', 'Research Scientist', 'Laboratory Technician', 'Manufacturing Director', 'Healthcare Representative', 'Manager', 'Sales Representative', 'Research Director', or 'Human Resources'
* `JobSatisfaction`: the employee's ranking of their satisfaction with their job. Minimum is 1, maximum is 4.
* `MaritalStatus`: 'Single', 'Married', or 'Divorced'
* `MonthlyIncome`: in dollars
* `NumCompaniesWorked`: the total number of companies the employee has worked at in their lifetime
* `OverTime`: whether or not the employee is eligible for overtime. 1 (yes) or 0 (no)
* `PercentSalaryHike`: for the employee's most recent salary increase, the percentage by which their salary increased
* `WorkLifeBalance`: employee's rating of their work-life balance. 1 (bad), 2 (good), 3 (better), 4 (best)
* `YearsAtCompany`: number of years employee has worked for IBM
* `YearsInCurrentRole`: number of years employee has worked for IBM in their current position

## 9. Machine Learning <a id= "section9"></a>

*Machine learning* is, essentially, pattern-matching. The process involves giving data to a computer, instructing it to find patterns within that data ("learn"), and then to perform a task based on what it has learned. We've already done machine learning in this course, when we trained a linear regression model in notebooks 07 and 08.

The bike-sharing problem from those notebooks and the attrition problem are both examples of **supervised learning**. In both cases, the data used to train the model have *labels*: each row in the data set includes the observed value of the response variable. For example, when we tried to predict bike ridership, we had the actual number of riders. Doing supervised learning is like studying with an answer key- you can use the key to check your answers, which can help you correct your mistakes and learn more quickly and accurately.

There are two main kinds of problems solved in supervised learning:
* **Regression problems** predict continuous amounts, as in the bike-sharing problem (predict number of riders based on features like temperature or day of the week). Another example would be predicting the optimal used car price based on its year, make, model, and mileage.
* **Classification problems** predict discrete categories or labels, as in our problem today (predicting whether or not an employee will leave a company). Another example would be labeling an image as "cat" or "dog" based on the image features.

<div class="alert alert-info">
    <p>
    <b>NOTE:</b> the other main category of machine learning is <b>unsupervised learning</b>, where we are trying to predict an amount or a category but we <i>don't</i> know the actual values. Unsupervised learning is a bit like trying to do a homework problem without an answer key, where you also don't know exactly what the problem is.   
    </p>
    <p>Common unsupervised learning algorithms include <i>clustering</i> and <i>neural networks</i>. 
    </p>
    </div>

### 9a. The K-Nearest Neighbors Algorithm <a id="section9a"></a>

Imagine you are given a simple classification problem. In this problem, there are two possible classes: red triangles and blue squares.

The next cell will create a plot of the data you're given to start out.

In [ ]:
# run this cell to show the toy data
plt.scatter(x=[1, 2, 3], y=[2, 4, 3], c='r', marker='^', s=100)
plt.scatter(x=[4, 5, 3], y=[3, 1, 1], c='b', marker='s', s=100);

Now imagine you are given a new data point to classify. You don't know the correct category, but this new point must be either a red triangle or a blue square. We will represent this unknown new data point with a black circle.

In [ ]:
# add the new data point
plt.scatter(x=[1, 2, 3], y=[2, 4, 3], c='r', marker='^', s=100)
plt.scatter(x=[4, 5, 3], y=[3, 1, 1], c='b', marker='s', s=100)
plt.scatter(x=2, y=2, c='k', s=100);

Do you think it's more likely that the new black point is a red triangle or a blue square?

If you're like most people, in the absence of any other information you may have used the **K-Nearest Neighbors** algorithm.

K-Nearest Neighbors (or KNN) picks a class for the new point based on the classes of the closest points- its "neighbors". In this algorithm, "closest" is referring to *Euclidean distance*: the shortest straight-line distance between two points.

The "k" in KNN refers to the number of neighbors that "vote" on the new point's classification. For example, if k=1, the algorithm would find the point closest to the new point...

In [ ]:
# show the 1 nearest neighbor
plt.scatter(x=[1, 2, 3], y=[2, 4, 3], c='r', marker='^', s=100)
plt.scatter(x=[4, 5, 3], y=[3, 1, 1], c='b', marker='s', s=100)
plt.plot([2, 1], [2, 2], 'r--', linewidth=1)
plt.scatter(x=2, y=2, c='k', s=100);

...and conclude that the new point should be a red triangle, because its neighbor is a red triangle.

If instead k=3, the algorithm would look for the three points closest to the new point...

In [ ]:
# show the 3 nearest neighbors
plt.scatter(x=[1, 2, 3], y=[2, 4, 3], c='r', marker='^', s=100)
plt.scatter(x=[4, 5, 3], y=[3, 1, 1], c='b', marker='s', s=100)
plt.plot([2, 3], [2, 3], 'r--', linewidth=1)
plt.plot([2, 1], [2, 2], 'r--', linewidth=1)
plt.plot([2, 3], [2, 1], 'b--', linewidth=1)
plt.scatter(x=2, y=2, c='k', s=100);

...and if all three points are weighted equally, it would decide the new point is a red triangle since the majority of the three neighbors are red.

The value of 'k' can make a big difference in the accuracy of the algorithm. Imagine you were asked to classify this new black point:

In [ ]:
# add the new data point
plt.scatter(x=[1, 2, 3], y=[2, 4, 3], c='r', marker='^', s=100)
plt.scatter(x=[4, 5, 3], y=[3, 1, 1], c='b', marker='s', s=100)
plt.scatter(x=3, y=2.5, c='k', s=100);

<div class="alert alert-warning">
    <b>QUESTION:</b> Using the KNN algorithm, how would this point be classified for $k=1$, $k=2$, and $k=3$? Why could it be a problem to have $k$ be an even number in this case?
    </div>

YOUR ANSWER HERE

### 9b. Using Scikit-Learn: An Example<a id="section9b"></a>

[Scikit-Learn](https://scikit-learn.org/stable/index.html) is free, open-source software for machine learning in Python. The software includes tools to create, train, and evaluate machine learning models.

One of the nice things about Scikit-Learn is that the models all have similar frameworks. If you learn how to train a linear regression model, for example, you can train a classification model by using almost identical code. The general steps are:

1. Choose your explanatory variables (what you're going to use to make predictions) and response variable (what you want to predict) 
1. Split the data into training and testing sets (and, if you're doing a lot of parameter tuning, a validation set)
2. Create a model in Python
3. Fit the data to the model
4. Make predictions using your fitted model
5. Score the accuracy of your model

In Notebooks 07 and 08, we implemented a linear regression model by writing out most of the code by hand. That's a great way to do things while you're learning, but in the future it would be nice to speed things up. Thankfully, Scikit-Learn can help. In this section, we'll show how to create the linear regression model with Scikit-Learn. In the next section, you'll follow the same steps to create a KNN model.

First, we **choose our explanatory and response variables**. The next cells load the bike data, select "temp", "weather", and "month" as the explanatory variables and "total riders" as the response variable, and create a new X DataFrame and y array that only contain our chosen variables.

In [ ]:
# run this cell to load the data
bikes = pd.read_csv("data/day_renamed_dso.csv")

# choose explanatory and response variables
expl_vars_bikes = ["temp", "weather", "month"]

resp_var_bikes = "total riders"

In [ ]:
# create the X DataFrame
X_bikes = bikes.loc[:, expl_vars_bikes]

# show the first 5 rows
X_bikes.head()

In [ ]:
# create the y array
y_bikes = bikes[resp_var_bikes]

# show the first 5 items
y_bikes.head()

Next, we **split the data into train and test sets**. This code looks a lot like the code from Notebook 07. Running the cell will put 80% of the data into a training set and the remaining 20% into a test set.

<div class="alert alert-info">
    <b>NOTE</b>: we're not using a validation set here for simplicity, and because we're not adjusting model parameters. If we were going through a full model selection process, including experimenting with what features to use in our model, we would absolutely want to use a validation set as in Notebook 08.
    </div>

In [ ]:
# split the data into training and test sets
X_train_bike, X_test_bike, y_train_bike, y_test_bike = train_test_split(X_bikes, y_bikes, 
                                                                        train_size=0.8, test_size=0.2)

Step three is to **create the model**. The next cell imports the LinearRegression model code from Scikit-Learn, then creates a new model.

A Scikit-Learn model is a Python *object*. To work with a Python object, it can help to conceive of it like a real-life object. Once a real-life object (like a cell phone) is created, it has a set of attributes associated with it (like the manufacturer, model, and storage capacity) and a set of functions it can perform (play music, send email, look up directions...).

Once a Scikit-Learn model is created (on the cloud server), it has a set of attributes associated with it (like the kind of algorithm it uses and whether or not it normalizes data) and a set of functions it can perform (its methods: it can be fitted to a data set, make predictions, and evaluate the accuracy of its predictions). Another example is the DataFrame object that we've been working with for several notebooks. DataFrames have attributes like shape and column names, and they have many functions they can perform, like selecting columns, filtering rows, and manipulating values.

After the model code is imported, we can create a new model. The arguments inside the model creation function call set some of the *hyperparameters*- features of the model that are chosen regardless of the data. In this case, we're using the defaults, which fit an intercept term and don't normalize the training data.

<div class="alert alert-info">
<b>NOTE:</b> we have included the <code>fit_intercept</code> and <code>normalize</code> arguments here to make it clear that hyperparameters are being set when you create the model. Because we are setting the hyperparameters to their default values, the code in the next cell will create exactly the same base model as if we had included no arguments at all (i.e. if the expression was <code>LinearRegression()</code>)</div>

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression(fit_intercept=True, normalize=False)


Now, the "raw" model can be fit to the training data using the `fit` method. This method takes two arguments: the features in the training X matrix and the labels as listed in the training y array.

In [ ]:
lr_model.fit(X_train_bike, y_train_bike)

What just happened? 

Behind the scenes, the `fit` method carried out all the steps we went through by hand in Notebook 07: adding an intercept term and using ordinary least squares to calculate $\beta$.

Once the model is fit, it has "learned" how to predict total riders based on the features we gave it (month, temperature, and weather). If it is given a matrix of those three features in its `predict` method, it will give back an array of total rider prediction numbers for each row in the matrix.

Using `predict` is the equivalent of taking the dot product by hand, as we did in Notebook 07 using `@`.

In [ ]:
# save the predictions to a variable
lr_predictions = lr_model.predict(X_test_bike)
# show the predictions
lr_predictions

Additionally, our model has a `score` method, which gives information about how well it is doing its prediction task. The `score` method takes as arguments a matrix of feature data (with columns temperature, weather, and month, since those are what it's been trained on) and an array of labels so it can check its answers. The number it returns is a bit different for different types of models. For linear regression, the number is the *coefficient of determination*, aka $r^2$. You can double-check what `score` returns by checking out the [Scikit-Learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)- scroll down to see the list of methods and read more abotu `score`.

To sum up, the best possible value of $r^2$ is 1 and the worst is 0.

In [ ]:
# save the score to a variable
lr_score = lr_model.score(X_test_bike, y_test_bike)
# show the score
lr_score

### 9c. Using Scikit-Learn: KNN <a id="section9c"></a>

Now it's time to return to our IBM employee attrition data set, to see if we can use a k-nearest neighbors model to predict whether or not an employee will leave the company. This section will follow the steps in section 9b very closely- if you're stuck, check the LinearRegression code for some hints.

As a reminder, here's what our attrition data looks like:

In [ ]:
# the first five rows of the attrition data
attrition.head()

Remember, our steps are:

1. Choose your explanatory variables (what you're going to use to make predictions) and response variable (what you want to predict) 
1. Split the data into training and testing sets 
2. Create a model in Python
3. Fit the data to the model
4. Make predictions using your fitted model
5. Score the accuracy of your model

Start by choosing explanatory and response variables. For this exercise, we'll try predicting attrition based on **monthly income**, **number of years in current role**, and **overtime eligibility**.

<div class="alert alert-warning">
    <b>EXERCISE</b>: Choose explanatory and response variables.
    <ul>
        <li>explanatory variables must be numerical feature column names that could help predict attrition. For this exercise, we'll be using the columns with data on the employee's monthly income, years in current role, and overtime status. The names should be strings (i.e. inside quotation marks), listed within the square brackets and separated by commas</li>
        <li>the response variable should be the name of the column that says whether or not an employee left by attrition, inside quotation marks</li>
    </ul>
    </div>

In [ ]:
# choose explanatory and response variables
expl_vars_attrition = [...]

resp_var_attrition = ...

In [ ]:
# run this cell to check your answer for some common mistakes
check("tests/vars.ok")

Now, split the data into training and test sets. If you've chosen your variables correctly, you can just run the next few cells. The training data will be called `X_train_att` and `y_train_att`, while the test data will be called `X_test_att` and `y_test_att`.

In [ ]:
# once you've chosen your variables, just run this cell
# create the X DataFrame
X_attrition = attrition.loc[:, expl_vars_attrition]

# show the first 5 rows
X_attrition.head()

In [ ]:
# once you've chosen your variables, just run this cell
# create the y array
y_attrition = attrition[resp_var_attrition]

# show the first 5 items
y_attrition.head()

In [ ]:
# once you've chosen your variables, just run this cell
# split the data into training and test
X_train_att, X_test_att, y_train_att, y_test_att = train_test_split(X_attrition, y_attrition,
                                                                    train_size=0.8, test_size=0.2)

The next step is to create the KNN model. The first line of the next cell imports the code for the model, which is called `KNeighborsClassifier`. For now, we'll use the default number of neighbors, which is 5.

<div class="alert alert-warning">
    <b>EXERCISE:</b> Create the KNN model. Replace the ellipses with a model creation call expression, which is the model type followed by parentheses (don't put anything in the parentheses to use the default number of neighbors). We will assign the created model the name of <code>knn</code> using an equals sign.
    </div>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# create the KNeighborsClassifier
knn = ...


In [ ]:
# run this cell to check your answer for some common mistakes
check("tests/model.ok")

With the model created, it can now be fit. 

<div class="alert alert-warning">
    <b>EXERCISE:</b> Fit the KNN model to the training data. Use the <code>fit</code> method on the model to train it. <code>fit</code> takes two arguments: the training feature matrix (<code>X_train_att</code>) and the list of classes for the training data (<code>y_train_att</code>).
    </div>

In [ ]:
# fit the knn model to the training data
# look at the code for the linear regression model for a hint
...

A fitted model can make predictions. Use the fitted model's `predict` to predict attrition for the test data. `predict` takes only one argument: an X matrix of feature data with the same features (columns) that it was trained on.

<div class="alert alert-warning">
    <b>EXERCISE:</b> Use <code>knn.predict</code> to make predictions for the test data (<code>X_test_att</code>). Check the code for the linear regression model predictions for a hint on how the code expression should look.
    </div>

In [ ]:
# use the fitted knn model to predict attrition for the test data
# look at the code for the linear regression model for a hint
knn_predictions = ...
knn_predictions

In [ ]:
# run this cell to check your answer for some common mistakes
check("tests/pred.ok")

Finally, let's evaluate the model. 

For KNN, the `score` method [returns the mean accuracy on the data and labels](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.score). Put another way, the score for KNN is the proportion of employees correctly labeled by the model- the number of correctly labeled employees divided by the total number of employees. So, a score of 1.0 indicates a perfect model, and a score of 0.0 indicates a model that got all classifications wrong.

<div class="alert alert-warning">
    <b>EXERCISE:</b> Use the model's <code>score</code> method to evaluate its accuracy on the test data. <code>score</code> requires 2 arguments: a matrix of feature data (X) and an array of response variables (y).
    </div>

In [ ]:
# use the fitted knn model to score predictions for the test data
# look at the code for the linear regression model for a hint
knn_score = ...
# show the score
knn_score

In [ ]:
# run this cell to check your answer for some common mistakes
check("tests/score.ok")

#### Visualization
When evaluating the KNN model, it can also be helpful to visualize the data so we can see the scope of the problem we're trying to solve. 

The following scatter plots plot our explanatory variables against each other. Every point represents a single employee, and the color and shape of the point indicate whether or not they left IBM: an orange circle indicates attrition and a blue triangle indicates the employee remained at IBM.

Because KNN assumes that points of the same class are near to each other, the ideal KNN scatter plot would have all points of the same type located next to each other on the plot. 

This plot shows monthly income and years in current role:

In [ ]:
# make the plots bigger
sns.set(rc={'figure.figsize':(11,8)})

# create the scatter plot
sns.scatterplot(x=attrition["MonthlyIncome"], 
                # add a bit of jitter to YearsInCurrentRole so the points overlap less
                y=attrition["YearsInCurrentRole"] + np.random.normal(scale = 0.3, size=attrition.shape[0]), 
                data=attrition,
                hue="Attrition", style="Attrition", markers={0:"^", 1:"o"}, palette={0:"#003262", 1:"#FDB515"});

Sometimes, adding a third dimension can make a difference for the KNN algorithm. The next plot shows all three of our chosen explanatory variables: monthly income, years in current role, and overtime. Note that overtime is a binary value (employees are either eligible for overtime or not), so the overtime axis splits the points into two planes (one for each possible value of "OverTime").

In [ ]:
# make the 3D scatter plot
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

att_pos = attrition[attrition.Attrition == 1]
att_neg = attrition[attrition.Attrition == 0]

x_label = "YearsInCurrentRole"
y_label = "MonthlyIncome"
z_label = "OverTime"

for data, m , l, c in [(att_neg, "^", "Attrition = 0", "#003262"), (att_pos, "o", "Attrition = 1", "#FDB515")]:
    ax.scatter(data[x_label], data[y_label], (data[z_label] + np.random.normal(scale=0.05,size=data.shape[0])), 
               marker=m, label=l, color=c)

ax.set_xlabel(x_label)
ax.set_ylabel(y_label)
ax.set_zlabel(z_label)

plt.legend();

<div class="alert alert-warning">
    <b>QUESTION:</b> based on the model score, how well is our classifier working? Based on the scatter plot, who are the people most likely to be mis-classified, and how can you tell (hint: think about how KNN makes predictions)?
    </div>

YOUR ANSWER HERE

#### References
- [IBM HR Analytics Employee Attrition & Performance mock data set](https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset/home) is made available under the [Open Database License](http://opendatacommons.org/licenses/odbl/1.0/). Any rights in individual contents of the database are licensed under the [Database Contents License](http://opendatacommons.org/licenses/dbcl/1.0/).